# Pengambilan Data

In [13]:
topik = 'bitcoin'
waktu_ambil = '15 maret malam'

In [14]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [15]:
import twitter_credentials
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

In [17]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

In [18]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [19]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
      
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                       tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [20]:
 class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
         
    def clean_tweet(self, tweet):
    
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w +:\/\/ \S +)", " ", tweet).split())
    
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        # analysis = analysis.translate(from_lang = 'in',to='en')

        if analysis.sentiment.polarity > 0:
            return "Positif"
        elif analysis.sentiment.polarity == 0:
            return "Netral"
        else:
            return "Negatif"
        
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        
        df['id'] = np.array([tweet.id_str for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['lang'] = np.array([tweet.lang for tweet in tweets])
        df['place'] = np.array([tweet.place for tweet in tweets ])
        

        return df

In [21]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()
    keyword = [topik]
    api = twitter_client.get_twitter_client_api()
    tweets = api.search(keyword, count=100, lang = 'en')

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df)

                                               tweets                   id  \
0   LONDON (Reuters) - Bitcoin plummeted on Thursd...  1239178374230278146   
1   When you take possession of me, it throws me r...  1239178367758413827   
2   RT @satoshi_babe: 💰 50$ Bitcoin Giveaway 💰\n\n...  1239178364344242177   
3   RT @Bloqport: Bitcoin yearly lows update:\n\n2...  1239178337731391489   
4         #Bitcoin fixes this https://t.co/NreU3UzP0h  1239178335168716800   
..                                                ...                  ...   
95  RT @satoshi_babe: 💰 50$ Bitcoin Giveaway 💰\n\n...  1239177622610939907   
96  RT @VentureCoinist: The newsletter from @Crypt...  1239177619154878465   
97  @CalvinAyre Mr @CalvinAyre we have to make BSV...  1239177602503507968   
98  RT @whale_alert: 1,100 #BTC (5,898,788 USD) tr...  1239177601719164928   
99  #Trending: Granite Geek: SNHU experiments with...  1239177588804915201   

                  date               source lang place sentimen

# Preproses

In [22]:
def remove_punct(text):
  
    #hapus simbol
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub('rt @[^\s]+',' ', text)
    text = re.sub('@[^\s]+',' ',text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')
    text  = "".join([char for char in text if char not in string.punctuation])
    #text = re.sub('[0-9]+', '', text)

    return text

df['tweet_bersih'] = df['tweets'].apply(lambda x: remove_punct(x.lower()))

In [23]:
#--- REMOVE DUPLICATES
df = df.drop_duplicates(['tweet_bersih'])
df

,tweets,id,date,source,lang,place,sentiment,tweet_bersih
0,LONDON (Reuters) - Bitcoin plummeted on Thursd...,1239178374230278146,2020-03-15 13:15:21,ContentStudio.io,en,None,Positif,london reuters bitcoin plummeted on thursday ...
1,"When you take possession of me, it throws me r...",1239178367758413827,2020-03-15 13:15:20,Twitter Web Client,en,None,Positif,when you take possession of me it throws me ri...
2,RT @satoshi_babe: 💰 50$ Bitcoin Giveaway 💰\n\n...,1239178364344242177,2020-03-15 13:15:19,Mobile Web (M2),en,None,Netral,50 bitcoin giveaway follow and retweet and ta...
3,RT @Bloqport: Bitcoin yearly lows update:\n\n2...,1239178337731391489,2020-03-15 13:15:13,Twitter Web App,en,None,Netral,bitcoin yearly lows update 2012 4 2013 65 201...
4,#Bitcoin fixes this https://t.co/NreU3UzP0h,1239178335168716800,2020-03-15 13:15:12,Twitter Web App,en,None,Netral,bitcoin fixes this
...,...,...,...,...,...,...,...,...
94,RT @StartGrowthHack: Time of each transaction ...,1239177623122608128,2020-03-15 13:12:22,Twitter for Android,en,None,Netral,time of each transaction by cryptocurrency fi...
96,RT @VentureCoinist: The newsletter from @Crypt...,1239177619154878465,2020-03-15 13:12:21,Twitter for Android,en,None,Netral,the newsletter from bitmex ceo hits different...
97,@CalvinAyre Mr @CalvinAyre we have to make BSV...,1239177602503507968,2020-03-15 13:12:17,Twitter for Android,en,None,Negatif,mr we have to make bsv independent of btc i m...
98,"RT @whale_alert: 1,100 #BTC (5,898,788 USD) tr...",1239177601719164928,2020-03-15 13:12:17,Twitter for iPhone,en,None,Negatif,1100 btc 5898788 usd transferred from unknown...


In [24]:
df.to_excel("Data Tanggal " + waktu_ambil + " (" + topik + ").xlsx")
df.to_csv("Data Tanggal " + waktu_ambil + " (" + topik + ").csv")